# 深層学習を用いた言語モデル

*Deep Learning*

深層学習を用いて言語モデルを作成する。  
前章で作成したマルコフモデルの様な「ある単語から次の単語を予測するモデル」をニューラルネットワークを用いて作成する。

本章では、深層学習を活用した言語モデル実装の基礎を学ぶ。

In [1]:
import random
from typing import List

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### データセット

wiki40b

In [3]:
text_path = 'data/jawiki.txt'
with open(text_path) as f:
    data = f.read().splitlines()

print('num of data:', len(data))
data[:3] # examples

num of data: 89699


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']

多すぎるので減らす

In [4]:
n_data = 2000
data = data[:n_data]

text_path = f'data/jawiki_{n_data}.txt'
with open(text_path, 'w') as f:
    f.write('\n'.join(data))


---

## サブワード分割

トークン化の手法。

NNでも、マルコフモデル同様、文章をトークン化して学習を行う。  
NNでは文字列をそのまま扱えないので、各トークンにID=クラスラベルを割り当てて、そのシーケンスとして文章を扱う。

前章とは異なり、サブワード分割という新たな手法を使う。ちゃんと理由もある。

1トークンあたりの文字数が多いことは、いくつかのデメリットを生む。  
例えば、トークンの種類が多くなること。深層学習を用いた言語モデルはトークンの種類に比例してパラメータの数が増え、学習が困難になる。  
それから、未知語が増えること。長いトークンは、それだけ多くの情報を持った限定的な言葉ということとなり、これらで語彙が埋まると表現力が落ちる。またデータセットに存在しない言葉を扱うことが困難になる。

サブワード分割は、これらの問題を解決する。  
サブワードと呼ばれる、単語よりも小さな単位に分割することで、1トークンあたりの文字数を減らす。

この手法では、データセットから頻出する文字の並びを学習し、その並びをトークンとして分割する。学習された、トークン化を行うモデルをトークナイザと呼ぶ。  
データセットに合ったトークン化が可能。また語彙数を指定することも可能。指定した語彙数に収まるまで細かく分割してくれる。

ちなみに、ChatGPTが使っているトークナイザはこれ: [OpenAI Platform](https://platform.openai.com/tokenizer)

少し余談。

前章でトークン化について以下のように説明した。

> トークンとはモデルが扱える最小単位のことで、例えば単語が該当する。

ここで、そもそも言語モデルは単語の確率を扱うものなので、絶対にトークン=単語にならないとおかしいとも考えられる。  
実は文章生成においては、単語を最小単位にしなければならない理由はない。極端な話、文字を最小単位として文章を文字の並びと見てもいい訳だ。なんらかのシーケンスとできればそれで充分なのだ。実際、現在有名な言語モデルの多くは単語を最小単位としていない。

ただ、言語モデルは単語の並びに確率を割り当てるモデルだ。単語を最小単位とするモデルだ。  
もしこの定義に厳格になるのであれば、現在有名な多くの言語モデルは言語モデルと呼べないのかもしれんね。しらんけど。

閑話休題。実際にやってみよう。  
sentencepieceというライブラリを用いる。

In [5]:
import sentencepiece as spm

### 学習

語彙数とデータセット（テキストファイル）を指定して、学習させる。

In [6]:
tokenizer_prefix = f'models/tokenizer_jawiki_{n_data}' # トークナイザのモデル名
n_vocab = 8000 # 語彙数
spm.SentencePieceTrainer.Train(
    input=text_path, # データセット
    model_prefix=tokenizer_prefix,
    vocab_size=n_vocab
)

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/jawiki_2000.txt
  input_format: 
  model_prefix: models/tokenizer_jawiki_2000
  model_type: UNIGRAM
  vocab_size: 8000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differ

### トークン化

学習したモデルにテキストを突っ込むとトークン化してくれる。

In [7]:
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model') # モデル読み込み
text = '今日はいい天気だ'
ids = sp.encode(text)
ids

[11, 1897, 6, 2436, 664, 287, 346]

テキストがトークン化され、ID列として取得できる。  
`out_type`を指定すると文字列のリストが取得できる。

In [8]:
sp.encode(text, out_type=str)

['▁', '今日', 'は', 'いい', '天', '気', 'だ']

アンダーバーは空白を意味する。初めにアンダーバーが付くのは仕様。

ID列を文字列に戻すこともできる。

In [9]:
sp.decode(ids)

'今日はいい天気だ'

全てのデータをトークン化（ID化）しよう。  
文章のリスト: `List[str]`を`sp.encode()`に与えるとID列のリスト: `List[List[int]]`が返ってくる。

In [10]:
data_ids = sp.encode(data)
n_vocab = len(sp)
print('num of vocabrary:', n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[11, 19, 477, 653, 323, 51, 570, 57, 3856, 1583]


---

## 特殊トークン

言語モデルを扱う際、データを扱いやすくするために特殊なトークンを考えることがある。  
以下に例を示す。

### *Unknown*

未知語を意味するトークン。  
推論時に学習データに含まれなかった単語に出会ったときに対応できるようになる。学習データ内での出現回数が少ない単語も未知語として扱うことがある。

### *Begin of Sentence*

文章の先頭を意味するトークン。  
先程の例では初めの単語を与える必要があったが、このトークンを作ることで、モデルに「文章の初め」を伝えられるようになる。

### *End of Sentence*

文章の終わりを意味するトークン。  
先程の例では句点が出た場合に生成を止めたが、本来句点は文の終わりであって文章の終わりではない。このトークンを作ることでモデルが文章の終わりを伝えられるようになる。

学習データの全ての文章の該当箇所（BOSであれば文章の初め、EOSであれば文章の終わり）にこれらのトークンを入れてから学習させることで、そのトークンの意味をモデルは理解する。  
またモデルが直接触れるのはトークンではなくトークンIDなので、特殊トークンの名前は何でもいい。他のトークンと重複しないように括弧を付けたりすることが多い。`[BOS]`とか`<EOS>`とか。

sentencepieceでは上記3つのトークンがID0, 1, 2にデフォルトで設定されている。

In [11]:
unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()
unk_id, bos_id, eos_id

(0, 1, 2)

bosとeosをデータに追加しておこう。

In [12]:
for ids in data_ids:
    ids.insert(0, bos_id) # 先頭にBOSを追加
    ids.append(eos_id) # 末尾にEOSを追加


---

## 学習データの作成

NNの学習を行うため、入力と出力のペアを作成する。  
今回はある単語から次に続く単語を予測するモデルを作成するので、ある単語IDとその次の単語IDがペアとなったデータを作成する。

In [13]:
Ids = List[int]
class TextDataset(Dataset):
    def __init__(self, data_ids: List[Ids]):
        self.x = [] # 入力
        self.t = [] # 正解
        for ids in data_ids:
            self.x += ids[:-1] # 単語
            self.t += ids[1:] # 次の単語
        self.n_data = len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.t[idx]

    def __len__(self):
        return self.n_data

# Dataset作成, 訓練データ:テストデータ = 8:2
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

# DataLoader作成
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# examples
x, y = next(iter(train_loader))
x, y

(tensor([ 930, 2904, 4035,   11, 2999,    8,  313,  451,    5, 1376,  102,  762,
           35,  374, 3384,   53,  570, 1223,  187,   91,  685, 5366, 3511,    3,
         7136, 3689, 1575, 4541,   11, 1289, 1434, 3582]),
 tensor([   6,   10,    4,  191, 5184,  805, 3316,    7, 2119, 6650,  284,   17,
           37, 4162,   17,   12,   39,    3,  135,   34,   32,   28,    5,   66,
          488, 1800,  115,    5,   62,  508,   15,  115]))


---

## 埋め込み層

*Embedding Layer*

指定したIDに対応するベクトルを出力する層。

単語IDはクラスラベルなので、そのままNNに入力するのは適切ではない。そこで、埋め込み層と呼ばれる層を用いて単語IDを指定した次元のベクトルに変換する。単語をベクトル化することは埋め込みと呼ばれるので、埋め込み層と呼ばれる。

埋め込み層は語彙の数だけベクトルを持っている。IDが入力されると、対応するベクトルが出力される。  
実装してみよう。

In [14]:
class Embedding(nn.Module):
    def __init__(self, n_vocab: int, embed_dim: int):
        super().__init__()
        self.vectors = torch.nn.Parameter(torch.randn(n_vocab, embed_dim))

    def forward(self, x):
        h = self.vectors[x]
        return h

embedding = Embedding(3, 5)

x = torch.tensor([0, 1, 2])
h = embedding(x)
h

tensor([[ 1.3393, -1.0283, -0.2885,  0.0278,  0.1828],
        [-0.7195,  0.6724, -0.7675, -0.4354,  0.3122],
        [-0.2893, -2.1155,  0.3440, -0.5355, -0.7502]],
       grad_fn=<IndexBackward0>)

`PyTorch`には`torch.nn.Embedding`というクラスが実装されているので、それを使うと良い。  
- https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

In [15]:
embedding = torch.nn.Embedding(3, 5)
h = embedding(x)
h

tensor([[ 0.1750, -0.9761,  0.2558, -1.1941,  1.0711],
        [-0.9422, -0.1731,  0.0815, -0.0630, -0.3444],
        [-1.6119,  0.0363,  0.6885, -0.9079,  0.9272]],
       grad_fn=<EmbeddingBackward0>)

埋め込み層が持っているベクトルは学習可能なパラメータである。  
埋め込み層が「one-hot化+線形変換」を行っていると見ればパラメータであることが理解しやすい。

また、言語モデルの学習によって得られる単語ベクトルは良い埋め込み表現として機能することが知られている。単語の埋め込み表現を得ることを目的として言語モデルを学習させる試みもあり、word2vecなどがこれに該当する。


---

## モデル構築

ある単語IDを入力に取り、次の単語IDを出力するモデルを作成する。

実際の出力は語彙数分の次元を持つベクトルとなる。  
つまりこのタスクは単語の分類問題とも言える。

モデルは埋め込み層と線形層で作る。最もシンプルな構成。

In [16]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab: int, hidden_size: int):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, hidden_size)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x):
        """
        x: (batch_size,)
        """
        h = self.embedding(x) # (batch_size, hidden_size)
        h = self.relu(h)
        y = self.fc(h) # (batch_size, n_vocab)
        return y

In [17]:
n_vocab = len(sp)
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size).to(device)

パラメータ数はこんな感じ。

In [18]:
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 8,200,000



---

## パープレキシティ

*Perplexity*

言語モデルの評価指標。

言語モデルは文脈から次の単語を予測する。単語はカテゴリ変数なので、言語モデルは単語に関する分類モデルとなる。

分類モデルでは、正解ラベルと予測ラベルの差を評価する指標として、交差エントロピーがよく用いられる。交差エントロピーは二つの確率分布の差を表す指標。

$$
H(p, q) = - \sum_{x} p(x) \log q(x)
$$

損失関数を交差エントロピーとしてNNを学習させることはよくある。予測された分布と正解の分布を与え、損失を計算する。

$$
L = H(t, y) = - \sum_i t_i \log y_i
$$

正解ラベルは基本的にone-hotベクトルなので、対応する部分だけ見れば良く、正解ラベルを$k$とすると以下のように表せる。

$$
L = - \log y_k
$$

負の対数尤度とも見られるね。

この交差エントロピーは当然言語モデルの学習でも使うことができる。実際に学習時の損失関数は交差エントロピーを使う。  
しかし、言語モデルを評価する際は、交差エントロピーによって求めた損失をそのまま見るのではなく、それを少し変形したパープレキシティを用いる。

$$
\text{ppl} = \exp L = \exp (- \log y_k) = \frac{1}{y_k}
$$

expoentialを取っただけ。結果的に「予測された正解単語の確率の逆数」になる。この値は**分岐数**とも呼ばれる。  
例えば、予測された正解単語の確率が1/2の場合、パープレキシティは2となる。これはモデルが単語を2択まで絞れているように解釈できる。分岐数が少なければ、モデル自信をもって正解単語を予測できていると解釈できる。  
このような解釈性が理由で、評価指標としてパープレキシティが用いられる。

ちなみに複数データの場合はこう。交差エントロピー部分が平均になる。

$$
\begin{align}
\text{ppl}
    &= \exp \left( - \frac{1}{N} \sum_{i=1}^N \log y_k^{(i)} \right) \\
    &= \prod_{i=1}^N \sqrt[N]{\frac{1}{y_k^{(i)}}}
\end{align}
$$

厳密には平均ではないが、各パープレキシティの平均として解釈しちゃってよい。相乗平均のようなイメージかな（相乗平均ともちょっと違うけど）。


---

## 学習

ではいよいよ学習させていく。  
損失関数に交差エントロピーを設定し、通常の分類モデルと同じように学習する。

In [19]:
loss_fn = nn.CrossEntropyLoss()

def eval_model(model):
    model.eval()
    losses = []
    with torch.no_grad():
        for x, t in test_loader:
            x = x.to(device)
            t = t.to(device)
            y = model(x)
            loss = loss_fn(y, t)
            losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(
        n_iter=len(train_loader),
        n_epochs=n_epochs,
        unit=prog_unit,
        label="ppl train"
    )
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            optimizer.zero_grad()
            x = x.to(device) # 入力
            t = t.to(device) # 正解
            y = model(x) # 出力
            loss = loss_fn(y, t) # 損失
            loss.backward() # 逆伝播
            optimizer.step() # パラメータ更新
            ppl = torch.exp(loss).item() # パープレキシティ
            prog.update(ppl) # 進捗バー更新

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f'test: {test_ppl:.5f}', no_step=True)
        prog.memo()

In [20]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [21]:
train(model, optimizer, n_epochs=20, prog_unit=2)

  1-2/20: ############################## 100% [00:01:37.75] ppl train: 876.91764, test: 634.09088 
  3-4/20: ############################## 100% [00:01:35.72] ppl train: 270.34231, test: 601.96765 
  5-6/20: ############################## 100% [00:01:36.44] ppl train: 192.06430, test: 630.06244 
  7-8/20: ############################## 100% [00:01:35.40] ppl train: 159.88602, test: 685.09576 
 9-10/20: ############################## 100% [00:01:36.31] ppl train: 142.92731, test: 764.88733 
11-12/20: ############################## 100% [00:01:34.97] ppl train: 132.34475, test: 861.72229 
13-14/20: ############################## 100% [00:01:32.03] ppl train: 125.50464, test: 983.70300 
15-16/20: ############################## 100% [00:01:32.25] ppl train: 120.48983, test: 1129.80676 
17-18/20: ############################## 100% [00:01:32.10] ppl train: 116.86110, test: 1310.66492 
19-20/20: ############################## 100% [00:01:32.40] ppl train: 113.93259, test: 1517.63538 


単語をピンポイントで予測するのは難しいので、テストデータに対する精度は低い様子。

In [22]:
model_path = "models/nn.pth"
torch.save(model.state_dict(), model_path)


---

## 文章生成

学習したモデルを用いて文章を生成する。

学習させたモデルは、ある単語から次の単語を予測するモデルである。厳密には、ある単語IDを入力に取り、次の単語IDを出力するモデルである。  
さらに厳密に言うと、出力は単語IDではなく確率分布である。この確率分布から次の単語IDをサンプリングすることで、次の単語を生成する。

実際に文章を生成させてみる。マルコフモデル同様、あるトークンからの次のトークンの予測を繰り返すことで文章を生成する。

初めのトークンはBOSとし、以下の条件を満たすまで単語の生成を続ける。
- EOSが生成される
- 単語数が指定した限度に達する

In [23]:
# モデルの出力から単語をサンプリングする関数
def token_sampling(y):
    """
    y: (1, n_vocab)
    """
    y.squeeze_(0) # (n_vocab,)
    y[unk_id] = -torch.inf # UNKがサンプリングされる確率を0にする
    probs = F.softmax(y, dim=-1) # 確率分布に変換
    token, = random.choices(range(n_vocab), weights=probs) # サンプリング
    return token

# 文章を生成する関数
def generate_sentence(model, max_len=50):
    model.eval()
    token_id = bos_id
    token_ids = []

    # 終了条件を満たすまで単語を生成
    while len(token_ids) <= max_len and token_id != eos_id:

        # 入力する単語IDをtensorに変換
        x = torch.tensor([token_id]).to(device)

        # 次の単語の確率分布を予測
        y = model(x)
        token_id = token_sampling(y) # サンプリング
        token_ids.append(token_id)

    sentence = sp.decode(token_ids)
    return sentence

In [24]:
model = LanguageModel(n_vocab, hidden_size).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [25]:
for _ in range(5):
    print(generate_sentence(model))

『ドラマが見つかったりだすごし原景法を禁止する地主成熟、ゲーリングにとって内最大、学園新田次郎は大きな損害するに残存がジョナ (ものではない。ive経由を開
195月の間にの犯を激怒り、マイケル・ラー的な物資である。産みにした「これらの複数のメール、マロリーは合衆国憲法違反(八幡宮原始末社できることが特筆」を意味する。
and theyd.)と戦オランダ・カトリック教会の同一家の統合戦線戦へのしょう。
19世紀末にまでに敗北の病院に日にませきた。王潮ら手打ちが始まる王セルティック・ブなさせており、正化等は様?』(監督・カリフ・テレビ・DVD販売店、4
は社は主にホイナ=カストリ湾では、UShe 2に電話である(全国の選手で修正式の11月6日)に伝えられている。また、さし、ウルデ=ル人


マルコフモデル同様、直前の単語のみを予測に用いているため、不自然な文章が多く生成される。

出来上がったモデルは優れたものではないが、深層学習を用いた言語モデル実装の基礎を学ぶことができた。